In [82]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
import gmaps

# Census & gmaps API Keys
from config import (census_api_key, gkey)
c = Census(census_api_key, year=2017)

# Configure gmaps
gmaps.configure(api_key=gkey)


In [83]:
# Pull commuting data (counts and minutes) from Census API by state. Looks at people who work over the age of 16.
commute_byst= c.acs5.get(("B08136_003E", "B08136_004E", "B08136_007E","B08136_011E", "B08136_012E","B08301_001E",
                           "B08301_003E","B08301_004E", "B08301_010E","B08301_019E","B08101_041E","NAME"), {'for': 'state:*'})

# Convert to DataFrame
commute_byst_pd = pd.DataFrame(commute_byst)
                                
commute_byst_pd = commute_byst_pd.rename(columns={"B08136_003E":"count_solo_auto", "B08136_004E":"count_carpool", "B08136_007E":"count_pub_trans",
                                                    "B08136_011E":"count_walked", "B08136_012E":"count_other","B08301_001E":"time_total","B08301_003E":"time_solo_auto","B08301_004E":"time_carpool",
                                                    "B08301_010E":"time_public_transport","B08301_019E":"time_walked","B08101_041E":"time_other","NAME": "Name"})
commute_byst_pd.head()



,time_other,count_solo_auto,count_carpool,count_pub_trans,count_walked,count_other,time_total,time_solo_auto,time_carpool,time_public_transport,time_walked,Name,state
0,19331.0,24161705.0,2761480.0,1057085.0,393255.0,502590.0,1009568.0,828868.0,85272.0,21692.0,32317.0,Puerto Rico,72
1,20531.0,42207585.0,4900280.0,324595.0,241960.0,639890.0,2030100.0,1743217.0,173758.0,7433.0,22659.0,Alabama,01
2,18829.0,4423270.0,871085.0,219085.0,242730.0,733835.0,360701.0,247111.0,44744.0,5453.0,28714.0,Alaska,02
3,76546.0,54323095.0,8771110.0,2832275.0,694310.0,2286730.0,2916850.0,2234990.0,317969.0,58267.0,57699.0,Arizona,04
4,15329.0,22402830.0,3183235.0,159640.0,220485.0,396940.0,1258851.0,1044010.0,132343.0,4663.0,21933.0,Arkansas,05


# Is the ideal place to live "Nirvana" - not possible to find?
https://www.washingtonpost.com/transportation/2018/09/17/american-commutes-keep-getting-longer-according-survey-data-show/?noredirect=on&utm_term=.923e615cdc1d

https://www.census.gov/acs/www/about/why-we-ask-each-question/commuting/

## Share of commute time: main mode is "Public Transit": 
## DC and NY stand out
2017 Census Data:  Percentage of total commute time by main mode, all workers age 16+

In [84]:
pct_t_solo_auto= round((commute_byst_pd["time_solo_auto"]/commute_byst_pd["time_total"]*100),1)
pct_t_carpool= round((commute_byst_pd["time_carpool"]/commute_byst_pd["time_total"]*100),1)
pct_t_pub_trans= round((commute_byst_pd["time_public_transport"]/commute_byst_pd["time_total"]*100),1)
pct_t_walk= round((commute_byst_pd["time_walked"]/commute_byst_pd["time_total"]*100),1)
pct_t_other= round((commute_byst_pd["time_other"]/commute_byst_pd["time_total"]*100),1)
name=commute_byst_pd["Name"]

calc_commute_pd=pd.DataFrame({"Solo Auto": pct_t_solo_auto,"Carpool": pct_t_carpool,"Public Transit":pct_t_pub_trans,
                              "Walking": pct_t_walk, "Other": pct_t_other,"State":name})

sort_pubtrans_pd = calc_commute_pd.sort_values("Public Transit", ascending=False)
sort_pubtrans_pd = sort_pubtrans_pd[["State","Public Transit","Walking", "Solo Auto","Carpool","Other"]]

sort_pubtrans_pd.head(10)

,State,Public Transit,Walking,Solo Auto,Carpool,Other
8,District of Columbia,35.4,13.2,34.0,5.4,6.2
33,New York,28.2,6.3,52.9,6.6,1.9
31,New Jersey,11.5,3.0,71.5,8.0,1.8
21,Massachusetts,10.2,4.8,70.7,7.5,2.0
14,Illinois,9.4,3.0,73.3,8.0,1.7
22,Maryland,8.8,2.4,73.8,9.1,1.4
12,Hawaii,6.6,4.6,67.1,13.7,3.5
49,Washington,6.3,3.6,72.3,10.0,2.1
39,Pennsylvania,5.6,3.8,76.4,8.5,1.4
5,California,5.2,2.7,73.6,10.4,2.6


## Share of commute time when main mode is "Walking": 
## New states without much public transit join DC and NYC
2017 Census Data:  Percentage of total commute time by main mode, all workers age 16+

In [85]:
sort_walk_pd = calc_commute_pd.sort_values("Walking", ascending=False)
sort_walk_pd = sort_walk_pd[["State","Walking","Public Transit","Solo Auto","Carpool","Other"]]

sort_walk_pd.head()

,State,Walking,Public Transit,Solo Auto,Carpool,Other
8,District of Columbia,13.2,35.4,34.0,5.4,6.2
2,Alaska,8.0,1.5,68.5,12.4,5.2
33,New York,6.3,28.2,52.9,6.6,1.9
46,Vermont,5.6,1.2,75.9,8.9,1.6
27,Montana,5.1,0.8,75.6,9.7,2.4


## Under construction
## Share of commute time when main mode is "Walking": 
## New states without much public transit join DC and NYC

In [86]:
worker_commuting=commute_byst_pd["count_solo_auto"]+commute_byst_pd["count_carpool"]+commute_byst_pd["count_pub_trans"]+commute_byst_pd["count_walked"]+commute_byst_pd["count_other"]

worker_commuting_pd=pd.DataFrame({"Workers Commuting": worker_commuting})

#worker_commuting_pd.head()

combined_commute = pd.merge(worker_commuting_pd, commute_byst_pd, how='outer', on='state')




KeyError: 'state'